In [4]:
import leafmap
from tqdm import tqdm
import datetime
import geopandas as gpd
import pandas as pd
import os
from pathlib import Path

if Path.cwd().name != 'src':
    os.chdir('./src')

In [5]:
#Read the csv file
events_df = pd.read_csv('../metadata/dateEventi.csv', sep=';')

# Create a dictionary with the event name as key and the date as value
event2date = events_df.set_index('Aligned name')['date'].to_dict()

In [6]:
all_collections = leafmap.maxar_collections()
all_collections

['BayofBengal-Cyclone-Mocha-May-23',
 'Emilia-Romagna-Italy-flooding-may23',
 'Gambia-flooding-8-11-2022',
 'Hurricane-Fiona-9-19-2022',
 'Hurricane-Ian-9-26-2022',
 'Hurricane-Idalia-Florida-Aug23',
 'Iceland-Volcano_Eruption-Dec-2023',
 'India-Floods-Oct-2023',
 'Indonesia-Earthquake22',
 'Japan-Earthquake-Jan-2024',
 'Kahramanmaras-turkey-earthquake-23',
 'Kalehe-DRC-Flooding-5-8-23',
 'Libya-Floods-Sept-2023',
 'Marshall-Fire-21-Update',
 'Maui-Hawaii-fires-Aug-23',
 'McDougallCreekWildfire-BC-Canada-Aug-23',
 'Morocco-Earthquake-Sept-2023',
 'NWT-Canada-Aug-23',
 'Nepal-Earthquake-Apr-2015',
 'Nepal-Earthquake-Nov-2023',
 'New-Zealand-Flooding23',
 'SmokeHouseCreek-Wildfires-Texas-Mar24',
 'Sudan-flooding-8-22-2022',
 'afghanistan-earthquake22',
 'cyclone-emnati22',
 'ghana-explosion22',
 'kentucky-flooding-7-29-2022',
 'pakistan-flooding22',
 'shovi-georgia-landslide-8Aug23',
 'southafrica-flooding22',
 'tonga-volcano21',
 'volcano-indonesia21',
 'yellowstone-flooding22']

### Funzioni per controllare se le cartelle sono full

In [7]:
def get_true_img_set(collection_id, child_id):
    gdf = gpd.read_file(os.path.join('../metadata/from_github_maxar_metadata/datasets/', collection_id + '.geojson'))
    gdf = gdf[gdf['catalog_id'] == child_id]
    return set(gdf['quadkey'].values)

### Funzioni per downloadare eventi

In [8]:
def get_pre_post_gdf_local(collection_id, event2date = event2date, local_gdf = True):
    
    #Retrieve the event date
    try:
        event_date = event2date[collection_id] #'2023-10-04'
    except:
        print("ERROR: Event date not found!!!")
        return None, None

    #Create the geodataframe
    if local_gdf:
        gdf = gpd.read_file(os.path.join('../metadata/from_github_maxar_metadata/datasets/', collection_id + '.geojson'))
    else:
        gdf = gpd.GeoDataFrame()
        for child_id in tqdm(leafmap.maxar_child_collections(collection_id)):
            current_gdf = leafmap.maxar_items(
                collection_id = collection_id,
                child_id = child_id,
                return_gdf=True,
                assets=['visual'],
            )
        gdf = pd.concat([gdf, current_gdf])
        
    
    #Split the geodataframe
    pre_gdf = gdf[gdf['datetime'] < event_date]
    post_gdf = gdf[gdf['datetime'] >= event_date]

    print('Collection_id:',collection_id,'\nEvent date:', event_date)

    if pre_gdf.shape[0] + post_gdf.shape[0] == gdf.shape[0]:
        print("OK: All items are accounted for\n")
    else:
        print("ERROR: Some items are missing!!!\n")

    print("pre_gdf", pre_gdf.shape)
    print("post_gdf", post_gdf.shape)

    return pre_gdf, post_gdf

def download_event(collection_id, out_dir_root = "/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/"):
    
    pre_gdf, post_gdf = get_pre_post_gdf_local(collection_id)
    if pre_gdf is None or post_gdf is None:
        return

    leafmap.maxar_download(pre_gdf['visual'].to_list(), out_dir = os.path.join(out_dir_root, collection_id, 'pre', ""))
    leafmap.maxar_download(post_gdf['visual'].to_list(), out_dir = os.path.join(out_dir_root, collection_id, 'post', ""))
    


In [9]:
event2date

{'afghanistan-earthquake22': '2022-06-21',
 'BayofBengal-Cyclone-Mocha-May-23': '2023-05-23',
 'cyclone-emnati22': '2022-02-22',
 'Emilia-Romagna-Italy-flooding-may23': '2023-05-23',
 'Gambia-flooding-8-11-2022': '2022-08-09',
 'Hurricane-Fiona-9-19-2022': '2022-09-19',
 'Hurricane-Ian-9-26-2022': '2022-09-26',
 'Hurricane-Idalia-Florida-Aug23': '2023-08-29',
 'India-Floods-Oct-2023': '2023-10-04',
 'Indonesia-Earthquake22': '2022-11-21',
 'Kahramanmaras-turkey-earthquake-23': '2023-02-06',
 'Kalehe-DRC-Flooding-5-8-23': '2023-05-08',
 'kentucky-flooding-7-29-2022': '2022-07-29',
 'Libya-Floods-Sept-2023': '2023-09-12',
 'Marshall-Fire-21-Update': '2022-01-01',
 'Maui-Hawaii-fires-Aug-23': '2023-08-08',
 'Morocco-Earthquake-Sept-2023': '2023-09-09',
 'New-Zealand-Flooding23': '2023-01-27',
 'NWT-Canada-Aug-23': '2023-08-16',
 'pakistan-flooding22': '2022-07-26',
 'shovi-georgia-landslide-8Aug23': '2023-08-05',
 'southafrica-flooding22': '2022-04-13',
 'Sudan-flooding-8-22-2022': '2022-

In [21]:
ev_name = 'Hurricane-Ian-9-26-2022'
download_event(ev_name)

Collection_id: Hurricane-Ian-9-26-2022 
Event date: 2022-09-26
OK: All items are accounted for

pre_gdf (2509, 20)
post_gdf (1698, 20)
/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/Hurricane-Ian-9-26-2022/pre/10300100CBCC2400/120200031311.tif already exists. Skipping...
/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/Hurricane-Ian-9-26-2022/pre/10300100CBCC2400/120200031312.tif already exists. Skipping...
/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/Hurricane-Ian-9-26-2022/pre/10300100CBCC2400/120200031313.tif already exists. Skipping...
/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/Hurricane-Ian-9-26-2022/pre/10300100CBCC2400/120200031321.tif already exists. Skipping...
/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/Hurricane-Ian-9-26-2022/pre/10300100CBCC2400/120200031322.tif already exists. Skipping...
/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/Hurricane-Ian-9-26-2022/pre/10300100CBCC2400/120200031323.tif

In [22]:
from maxarseg.assemble import holders

event = holders.Event(ev_name,
                      seg_config = None,
                      det_config = None,
                      when='pre')

print(event.region_name)

all_mosaics_names = event.all_mosaics_names
print(all_mosaics_names)

m0 = event.mosaics[all_mosaics_names[0]]
print(f'Number of tiles: {m0.tiles_num}')

Creating event: Hurricane-Ian-9-26-2022
Region: USA
Mosaics: ['10300100D122D300', '10200100B01E7400', '10200100C0DA0700', '10400100703E5500', '10200100BC21AC00', '105001002591D500', '105001002908EA00', '10300100D0383500', '104001007D777200', '10200100C66ACB00', '10200100CA8D0500', '10200100CA0ED900', '105001002D0A8100', '10400100757D1800', '1040010076AFB000', '10300100CCC6CD00', '105001002D8BCC00', '10300100CD936D00', '10300100D1553D00', '10200100C8D7A600', '105001002D409800', '10300100CBCC2400', '1040010074CCEA00', '1050010028B99200', '10300100CE603100', '105001002757D500', '10200100B0214600', '104001007A5E4D00', '10300100D6CFC000', '1040010075B43400', '10300100D026E700', '10300100D01AC700', '105001002A491900', '1050010029A68300', '1040010072528400', '10200100C64F3000', '10300100D0965E00', '10200100B0344500', '10300100CB5B9E00', '10300100D1466F00', '10300100CCA6AB00', '1050010028AE6C00', '1040010078184300', '1040010074035100', '1040010076599E00', '10300100D8A7A100', '10500100274A7C00'

In [23]:
event.get_roi_polygon(True)

'POLYGON ((-83.8993045444558 22.173767305032644, -78.7549240662501 22.173767305032644, -78.7549240662501 33.97112424789772, -83.8993045444558 33.97112424789772, -83.8993045444558 22.173767305032644))'

### Funzioni non utilizzate downoladare event Cella con tutte le funzioni per download

In [35]:
def isChildCorrectlyDownloaded_(collection_id, child_id, out_dir_root = "/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/"):
    pre_gdf, post_gdf = get_pre_post_gdf(collection_id)
    if pre_gdf is None or post_gdf is None:
        return
    
    pre_childs = pre_gdf['catalog_id'].unique()
    post_childs = post_gdf['catalog_id'].unique()

    if child_id in pre_childs and child_id in post_childs:
        print('Child correctly downloaded')
        return True
    else:
        print('Child not correctly downloaded')
        return False

def get_pre_post_gdf_for_n_children_(collection_id, n = 1, event2date = event2date):
    
    out_dir_root = "/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/"

    #Retrieve the event date
    try:
        event_date = event2date[collection_id] #'2023-10-04'
    except:
        print("ERROR: Event date not found!!!")
        return None, None

    #Create the geodataframe
    gdf = gpd.GeoDataFrame()
    counter = 0
    for child_id in tqdm(leafmap.maxar_child_collections(collection_id)):
        if counter == n:
            break
        #check if the child is entirely downloaded
        if 
        current_gdf = leafmap.maxar_items(
            collection_id = collection_id,
            child_id = child_id,
            return_gdf=True,
            assets=['visual'],
        )
        gdf = pd.concat([gdf, current_gdf])
        counter += 1

def download_event_(collection_id):
    out_dir_root = "/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/"
    #if os.path.exists(os.path.join(out_dir_root, collection_id)):
    #    print('Already downloaded: ', collection_id)
    #    return

    pre_gdf, post_gdf = get_pre_post_gdf(collection_id)
    if pre_gdf is None or post_gdf is None:
        return
    
    leafmap.maxar_download(pre_gdf['visual'].to_list(), out_dir = os.path.join(out_dir_root, collection_id, 'pre', ""))
    leafmap.maxar_download(post_gdf['visual'].to_list(), out_dir = os.path.join(out_dir_root, collection_id, 'post', ""))

def reorganize_(collection_id):
    pre_gdf, post_gdf = get_pre_post_gdf(collection_id)

    pre_childs = pre_gdf['catalog_id'].unique()
    post_childs = post_gdf['catalog_id'].unique()

    print('Pre-event images:', pre_childs, '\nPost-event images:', post_childs)